In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import itertools
import collections

import gym
import tensorflow as tf

In [ ]:
import pdb

# Create bowl function

In [ ]:
def bowl_funct(x, y):
    return np.tanh( np.sqrt((4*x)**2 + (4*y)**2) -2 )

In [ ]:
x_min, x_max = -1, 1
y_min, y_max = -1, 1

x_space = np.linspace(x_min, x_max, num=100)
y_space = np.linspace(y_min, y_max, num=100)

Y, X = np.meshgrid(y_space, x_space)
Z = bowl_funct(X, Y)

In [ ]:
fig = plt.figure()
axis = fig.add_subplot(111, projection='3d')

axis.plot_wireframe(X, Y, Z)
axis.set_xlabel('x')
axis.set_ylabel('y')
axis.set_zlabel('z')

# Dataset

In [ ]:
train_x = np.stack([X.flatten(), Y.flatten()], axis=1)  # shape [10000, 2]
train_y = np.array(bowl_funct(train_x[:,0], train_x[:,1]), ndmin=2).T

In [ ]:
print('train_x.shape', train_x.shape)
print('train_y.shape', train_y.shape)

In [ ]:
def plot_data(data_x, data_y):
    assert data_x.ndim == 2 and data_x.shape[1] == 2
    assert data_y.ndim == 2 and data_y.shape[1] == 1
    
    fig = plt.figure()
    axis = fig.add_subplot(111, projection='3d')

    axis.scatter3D(data_x[:,0], data_x[:,1], data_y, marker='.', alpha=.05)
    axis.set_xlabel('x')
    axis.set_ylabel('y')
    axis.set_zlabel('z')

In [ ]:
def plot_data(data_x, data_y, axis=None):
    assert data_x.ndim == 2 and data_x.shape[1] == 2
    assert data_y.ndim == 2 and data_y.shape[1] == 1
    
    if axis is None:
        fig = plt.figure(figsize=[12,9])
        axis = fig.add_subplot(111, projection='3d')
    
    X = data_x[:,0].reshape([100, 100])
    Y = data_x[:,1].reshape([100, 100])
    Z = data_y.reshape([100,100])

    axis.plot_wireframe(X, Y, Z)
    axis.set_xlabel('x')
    axis.set_ylabel('y')
    axis.set_zlabel('z')
    
    # axis.view_init(30, -85)

In [ ]:
plot_data(train_x, train_y)

# Funct Approximator

In [ ]:
class TFFunctApprox():
    def __init__(self, nb_in, nb_hid_1, nb_hid_2, nb_out):

        # TF model
        assert nb_in == 2
        assert nb_hid_1 == 256
        assert nb_hid_2 == 256
        assert nb_out == 1
        
        self.trace = collections.defaultdict(list)
        
        try:    sess.close()
        except: pass
        tf.reset_default_graph()

        self._x = tf.placeholder(name='x', shape=[None, nb_in], dtype=tf.float32)
        self._y = tf.placeholder(name='y', shape=[None, nb_out], dtype=tf.float32)

        with tf.variable_scope('Hidden_1'):
            self._W_hid_1 = tf.get_variable('W_hid_1', shape=[nb_in, nb_hid_1], dtype=tf.float32)
            self._b_hid_1 = tf.get_variable('b_hid_1', shape=[nb_hid_1], dtype=tf.float32,
                                            initializer=tf.zeros_initializer())
            self._z_hid_1 = tf.matmul(self._x, self._W_hid_1) + self._b_hid_1
            self._h_hid_1 = tf.nn.relu(self._z_hid_1)

        with tf.variable_scope('Hidden_2'):
            self._W_hid_2 = tf.get_variable('W_hid_2', shape=[nb_hid_1, nb_hid_2], dtype=tf.float32)
            self._b_hid_2 = tf.get_variable('b_hid_2', shape=[nb_hid_2], dtype=tf.float32,
                                           initializer=tf.zeros_initializer())
            self._z_hid_2 = tf.matmul(self._h_hid_1, self._W_hid_2) + self._b_hid_2
            self._h_hid_2 = tf.nn.relu(self._z_hid_2)

        with tf.variable_scope('Output'):
            self._W_out = tf.get_variable('W_out', shape=[nb_hid_2, nb_out], dtype=tf.float32)
            self._b_out = tf.get_variable('b_out', shape=[nb_out],
                                         initializer=tf.zeros_initializer())
            self._y_hat = tf.matmul(self._h_hid_2, self._W_out) + self._b_out

        with tf.variable_scope('MSE'):
            self._mse = tf.reduce_mean( tf.pow(self._y - self._y_hat, 2) )

        self._optimizer = tf.train.RMSPropOptimizer(learning_rate=0.00025)
        #self._optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.025)
        grads_and_vars = self._optimizer.compute_gradients(self._mse)
        self._train_op = self._optimizer.apply_gradients(grads_and_vars)

        self._dW_hid_1 = grads_and_vars[0][0]
        self._db_hid_1 = grads_and_vars[1][0]
        self._dW_hid_2 = grads_and_vars[2][0]
        self._db_hid_2 = grads_and_vars[3][0]
        self._dW_out = grads_and_vars[4][0]
        self._db_out = grads_and_vars[5][0]
        
        self._sess = tf.Session()
        self._sess.run(tf.global_variables_initializer())
        
    def backward(self, x, y):
        _, y_hat, z_hid_2, z_hid_1, loss, \
            dW_hid_1, db_hid_1, dW_hid_2, db_hid_2, dW_out, db_out = self._sess.run(
                [self._train_op, self._y_hat, self._z_hid_2, self._z_hid_1, self._mse,
                self._dW_hid_1, self._db_hid_1, self._dW_hid_2, self._db_hid_2, self._dW_out, self._db_out],
                feed_dict={self._x: x, self._y:y})
        
        self.trace['loss'].append(loss)
        self.trace['z_hid_1'].append(z_hid_1)
        self.trace['z_hid_2'].append(z_hid_2)
        self.trace['z_out'].append(y_hat)
        
        self.trace['dW_hid_1'].append(dW_hid_1)
        self.trace['db_hid_1'].append(db_hid_1)
        self.trace['dW_hid_2'].append(dW_hid_2)
        self.trace['db_hid_2'].append(db_hid_2)
        self.trace['dW_out'].append(dW_out)
        self.trace['db_out'].append(db_out)
        
        W_hid_1, b_hid_1, W_hid_2, b_hid_2, W_out, b_out = self._sess.run(tf.trainable_variables())
        self.trace['W_hid_1'].append(W_hid_1)
        self.trace['b_hid_1'].append(b_hid_1)
        self.trace['W_hid_2'].append(W_hid_2)
        self.trace['b_hid_2'].append(b_hid_2)
        self.trace['W_out'].append(W_out)
        self.trace['b_out'].append(b_out)
        
        return y_hat, loss
    
    def forward(self, x):
        return self._sess.run(self._y_hat, feed_dict={self._x: x})

In [ ]:
tf.trainable_variables()

# Train Loop

In [ ]:
# n_in = 196   # 784
# n_hid = 128  # 128             # try 8, 128(def.), 2048
# n_out = 10
# lr = .03     # 0.03                # try 10, 1(best), 0.03, 0.0003

n_batch = 100
act_fun = 'relu'
completed_epochs = 0

# np.random.seed(0)

# Initialize weights
# var_hid = np.sqrt(1/n_in)             # try:  0.001,  sqrt(1/n_in),  1
# var_out = np.sqrt(1/n_hid)
# W_hid = np.random.normal(0.0, var_hid, [n_in, n_hid])
# W_out = np.random.normal(0.0, var_out, [n_hid, n_out])

app = TFFunctApprox(nb_in=2,
                    nb_hid_1=256,
                    nb_hid_2=256,
                    nb_out=1)
writer = tf.summary.FileWriter(logdir='tf_log', graph=app._sess.graph)
writer.flush()

In [ ]:
tf.trainable_variables()

In [ ]:
yy = app.forward(train_x)

In [ ]:
fig = plt.figure(figsize=[12,6])
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')
plot_data(train_x, train_y, ax1)
plot_data(train_x, app.forward(train_x), ax2)
plt.tight_layout()

In [ ]:
ti_ = 0
train_i = np.array(range(len(train_x)))
for e in range(10):
    print(e)
    
    if e % 1 == 0:
        fig = plt.figure(figsize=[12,3])
        ax1 = fig.add_subplot(121, projection='3d')
        ax2 = fig.add_subplot(122, projection='3d')
        plot_data(train_x, train_y, ax1)
        plot_data(train_x, app.forward(train_x), ax2)
        plt.tight_layout()
        plt.show()
    
    np.random.shuffle(train_i)
    
    for i in range(0, len(train_x), n_batch):

        # Get 128 sized batch, both as 2d arrays   
        batch = train_i[i:i+n_batch]
        x = train_x[batch]
        y = train_y[batch]
        
        y_hat, loss = app.backward(x, y)

        assert y_hat.shape == y.shape
        assert np.isscalar(loss)

        ti_ += 1
        
    completed_epochs += 1

# Loss

In [ ]:
tr_loss = np.array(app.trace['loss'])
print('tr_loss', tr_loss.shape)

In [ ]:
#
#   Plot Loss, Accuracy
#
fig, ax = plt.subplots(figsize=[12,6])
ax.plot(tr_loss, label='Mini-Batch loss', alpha=.5)

ax.legend()
ax.grid()

plt.tight_layout()
plt.show;

<br>
<br>
<br>
<br>
<br>

# Weight Plots

In [ ]:
import sys
sys.path.append('../Debug_NN')

In [ ]:
import plotann
import importlib
importlib.reload(plotann)

In [ ]:
tr_W_hid_1 = np.array(app.trace['W_hid_1'])
tr_b_hid_1 = np.array(app.trace['b_hid_1'])
tr_W_hid_2 = np.array(app.trace['W_hid_2'])
tr_b_hid_2 = np.array(app.trace['b_hid_2'])
tr_W_out = np.array(app.trace['W_out'])
tr_b_out = np.array(app.trace['b_out'])

tr_dW_hid_1 = np.array(app.trace['dW_hid_1'])
tr_db_hid_1 = np.array(app.trace['db_hid_1'])
tr_dW_hid_2 = np.array(app.trace['dW_hid_2'])
tr_db_hid_2 = np.array(app.trace['db_hid_2'])
tr_dW_out = np.array(app.trace['dW_out'])
tr_db_out = np.array(app.trace['db_out'])

print('tr_W_hid_1', tr_W_hid_1.shape, tr_W_hid_1.size/1e6)
print('tr_b_hid_1', tr_b_hid_1.shape, tr_b_hid_1.size/1e6)
print('tr_W_hid_2', tr_W_hid_2.shape, tr_W_hid_2.size/1e6)
print('tr_b_hid_2', tr_b_hid_2.shape, tr_b_hid_2.size/1e6)
print('tr_W_out', tr_W_out.shape, tr_W_out.size/1e6)
print('tr_b_out', tr_b_out.shape, tr_b_out.size/1e6)

print('tr_dW_hid_1', tr_dW_hid_1.shape, tr_W_hid_1.size/1e6)
print('tr_db_hid_1', tr_db_hid_1.shape, tr_b_hid_1.size/1e6)
print('tr_dW_hid_2', tr_dW_hid_2.shape, tr_W_hid_2.size/1e6)
print('tr_db_hid_2', tr_db_hid_2.shape, tr_b_hid_2.size/1e6)
print('tr_dW_out', tr_dW_out.shape, tr_W_out.size/1e6)
print('tr_db_out', tr_db_out.shape, tr_b_out.size/1e6)

In [ ]:
tr_z_hid_1 = np.array(app.trace['z_hid_1'])
tr_z_hid_2 = np.array(app.trace['z_hid_2'])
tr_z_out = np.array(app.trace['z_out'])
print('tr_z_hid_1', tr_z_hid_1.shape, tr_z_hid_1.size/1e6)
print('tr_z_hid_2', tr_z_hid_2.shape, tr_z_hid_2.size/1e6)
print('tr_z_out', tr_z_out.shape, tr_z_out.size/1e6)

### Hidden 1

In [ ]:
plotann.show_neurons_weights(tr_W_hid_1, tr_dW_hid_1, neurons=range(3),
                            title_prefix='Hidden_1', color='red', figsize=[16,4])

In [ ]:
plotann.show_biases(tr_b_hid_1+1e-6, tr_db_hid_1, title_prefix='Biases_1', color='red', figsize=[16,4])

In [ ]:
plotann.show_layer_summary(tr_W_hid_1, tr_dW_hid_1, title_prefix='Hidden_1', color='red', figsize=[16,5])

In [ ]:
es = 10000  # epoch_size
plotann.show_layer_activations(tr_z_hid_1, epoch_size=es, activation_function=act_fun,
                               title_prefix='Hidden_1', color=(1,0,0,1), figsize=[16,6])

In [ ]:
es = 10000  # epoch_size
plotann.show_neurons_activations(tr_z_hid_1, epoch_size=es, activation_function=act_fun, neurons=range(10),
                                 title_prefix='Hidden_1', color=(1,0,0,1))

### Hidden 2

In [ ]:
plotann.show_neurons_weights(tr_W_hid_2, tr_dW_hid_2, neurons=range(3),
                            title_prefix='Hidden_2', color='green', figsize=[16,4])

In [ ]:
plotann.show_biases(tr_b_hid_2+1e-6, tr_db_hid_2, title_prefix='Biases_2', color='green', figsize=[16,4])

In [ ]:
plotann.show_layer_summary(tr_W_hid_2, tr_dW_hid_2, title_prefix='Hidden_2', color='green', figsize=[16,5])

In [ ]:
es = 10000  # epoch_size
plotann.show_layer_activations(tr_z_hid_2, epoch_size=es, activation_function=act_fun,
                               title_prefix='Hidden_2', color=(0,1,0,1), figsize=[16,6])

In [ ]:
es = 10000  # epoch_size
plotann.show_neurons_activations(tr_z_hid_2, epoch_size=es, activation_function=act_fun, neurons=range(10),
                                 title_prefix='Hidden_2', color=(0,1,0,1))

### Output

In [ ]:
plotann.show_neurons_weights(tr_W_out, tr_dW_out, neurons=range(1),
                             title_prefix='Output', color='blue', figsize=[16,4])

In [ ]:
plotann.show_biases(tr_b_out+1e-6, tr_db_out, title_prefix='Bisases_Output', color='blue', figsize=[16,4])

In [ ]:
plotann.show_layer_summary(tr_W_out, tr_dW_out, title_prefix='Output', color='blue', figsize=[16,5])

In [ ]:
es = 10000  # epoch_size
plotann.show_layer_activations(tr_z_out, epoch_size=es, activation_function=act_fun,
                               title_prefix='Output', color=(0,0,1,1), figsize=[16,6])

In [ ]:
es = 10000  # epoch_size
plotann.show_neurons_activations(tr_z_out, epoch_size=es, activation_function=act_fun, neurons=range(tr_z_out.shape[-1]),
                                 title_prefix='Output', color=(0,0,1,1))

<br>
<br>
<br>
<br>
<br>